In [2]:
import numpy as np

# CONFIGURATION
POPULATION_SIZE = 20
NUM_ANTENNAS = 5
NUM_GENES = NUM_ANTENNAS * 2
GRID_SIZE = 6
NUM_GENERATIONS = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.1
TOURNAMENT_SIZE = 3
POSITION_BOUNDS = (1, GRID_SIZE)

def calculate_fitness(chromosome):
    positions = chromosome.reshape(NUM_ANTENNAS, 2)
    covered_blocks = set()

    min_coord, max_coord = POSITION_BOUNDS

    for x, y in positions:
        covered_blocks.add((x, y))

        neighbors = [
            (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)
        ]

        for nx, ny in neighbors:
            if (min_coord <= nx <= max_coord) and (min_coord <= ny <= max_coord):
                covered_blocks.add((nx, ny))

    fitness = len(covered_blocks)
    return fitness, 0.0

def initialize_population(pop_size, num_genes, bounds):
    lower, upper = bounds
    return np.random.randint(lower, upper + 1, size=(pop_size, num_genes))

def select_parents(population, fitnesses):
    selected = []
    for _ in range(len(population)):
        indices = np.random.choice(len(population), TOURNAMENT_SIZE, replace=False)
        tournament_fitnesses = fitnesses[indices]
        winner_relative_idx = np.argmax(tournament_fitnesses)
        winner_global_idx = indices[winner_relative_idx]
        selected.append(population[winner_global_idx])
    return np.array(selected)

def crossover(parent1, parent2):
    point = 6

    if np.random.rand() < CROSSOVER_RATE:
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    else:
        return parent1.copy(), parent2.copy()

def mutate(chromosome, bounds):
    lower, _ = bounds
    mutated_chromosome = chromosome.copy()

    for i in range(1, NUM_GENES, 2):
        current_y = mutated_chromosome[i]
        mutated_chromosome[i] = max(current_y - 1, lower)

    return mutated_chromosome

def run_genetic_algorithm():
    print("Starting 5-Antenna GA")
    print(f"Generations: {NUM_GENERATIONS}, Population Size: {POPULATION_SIZE}")
    print("-" * 35)

    population = initialize_population(POPULATION_SIZE, NUM_GENES, POSITION_BOUNDS)

    best_chromosome = None
    best_fitness = -np.inf

    for generation in range(NUM_GENERATIONS):
        results = [calculate_fitness(c) for c in population]
        fitnesses = np.array([res[0] for res in results])

        current_best_idx = np.argmax(fitnesses)
        current_best_fitness = fitnesses[current_best_idx]
        current_best_chromosome = population[current_best_idx].copy()

        if current_best_fitness > best_fitness:
            best_fitness = current_best_fitness
            best_chromosome = current_best_chromosome.copy()

        if (generation + 1) % 10 == 0 or generation == 0 or generation == NUM_GENERATIONS - 1:
            print(f"Gen {generation+1:03d}: Best Fitness = {best_fitness:d}")

        next_population = []
        next_population.append(current_best_chromosome) # Elitism

        parents = select_parents(population, fitnesses)

        i = 0
        while len(next_population) < POPULATION_SIZE:
            parent1 = parents[i]
            parent2 = parents[i+1]

            child1, child2 = crossover(parent1, parent2)

            if np.random.rand() < MUTATION_RATE:
                child1 = mutate(child1, POSITION_BOUNDS)
            if np.random.rand() < MUTATION_RATE:
                child2 = mutate(child2, POSITION_BOUNDS)

            next_population.append(child1)
            if len(next_population) < POPULATION_SIZE:
                next_population.append(child2)

            i += 2

        population = np.array(next_population)

    final_positions = best_chromosome.reshape(NUM_ANTENNAS, 2)
    print("-" * 35)
    print("Optimization Complete.")
    print(f"Final Best Fitness: {best_fitness:d}")
    print("Optimal Antenna Positions (x, y):")

    formatted_positions = [f"({x},{y})" for x, y in final_positions]
    print(formatted_positions)


np.random.seed(42)
run_genetic_algorithm()

Starting 5-Antenna GA
Generations: 100, Population Size: 20
-----------------------------------
Gen 001: Best Fitness = 21
Gen 010: Best Fitness = 21
Gen 020: Best Fitness = 21
Gen 030: Best Fitness = 21
Gen 040: Best Fitness = 21
Gen 050: Best Fitness = 21
Gen 060: Best Fitness = 21
Gen 070: Best Fitness = 21
Gen 080: Best Fitness = 21
Gen 090: Best Fitness = 21
Gen 100: Best Fitness = 21
-----------------------------------
Optimization Complete.
Final Best Fitness: 21
Optimal Antenna Positions (x, y):
['(2,2)', '(5,6)', '(1,5)', '(6,4)', '(4,4)']
